In [130]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV



In [131]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [132]:
# Path to the CSV file
from google.colab import drive
csv_file_path = '/content/drive/MyDrive/one_hot_encoded.csv'  # Replace with the actual path to your CSV file
# Read the CSV file

mydata = pd.read_csv(csv_file_path)

mydata = mydata.dropna()


In [133]:
X = mydata.drop(columns=['claim'])
y = mydata['claim']


# X = data.drop('PremiumPrice', axis=1)
# y = data['PremiumPrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=69)
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled = scaler.transform(X)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

ValueError: could not convert string to float: 'NoDisease'

In [ ]:
# # Create an instance of XGBRegressor with the given parameters
model = xgb.XGBRegressor(
        tree_method='hist',  # Comment out if no GPU is available
        max_depth=32,
        learning_rate=0.03,
        n_estimators=475,
        subsample=0.65,
        colsample_bytree=0.85,
        random_state = 69)
# possibleParameters= {
#   'max_depth' : range (2, 51, 1),
#   'n_estimators'  : range (50,1000,25),
#   'learning_rate' : [0.01, 0.02,0.03,0.05,0.1,0.2],
#   'subsample':  [0.65,0.7,0.75,0.8,0.85,0.95,1],
#   'colsample_bytree':[0.80,0.85,0.90,0.95,1],
# }


# rand_search = RandomizedSearchCV(
#     xgb.XGBRegressor(random_state=69),
#     param_distributions=possibleParameters,
#     n_iter=300,  # Set the number of iterations to 1
#     cv=5, n_jobs=-1
# )
# rand_search.fit(X_train_scaled, y_train)

# best_params = rand_search.best_params_

# print("Best Parameters:", best_params)


# #best parameters


# #predictions using the best model
# model = rand_search.best_estimator_
# Fit the model to the training data
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)
print(y_pred)

In [ ]:
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(mae)
print(math.sqrt(mse))

In [ ]:
# from sklearn.ensemble import BaggingRegressor
# bagging_model = BaggingRegressor(
#     base_estimator=model,
#     n_estimators=10,       # Number of bootstrap samples
#     bootstrap=True,        # Use bootstrapped samples
#     n_jobs=-1,
#     random_state = 69# Use all available cores
# )

In [ ]:
# bagging_model.fit(X_train_scaled, y_train)

In [ ]:
# y_pred2 = bagging_model.predict(X_test_scaled)

In [ ]:
# mae = mean_absolute_error(y_test, y_pred2)
# mse = mean_squared_error(y_test, y_pred2)
# print(mae)
# print(math.sqrt(mse))

In [ ]:
# import numpy as np
# from sklearn.model_selection import cross_val_score, KFold
# kf = KFold(n_splits=10, shuffle=True, random_state=69)
# cv_mae = []
# cv_mse = []

# for train_index, test_index in kf.split(X_scaled):
#     X_train, X_test = X_scaled[train_index], X_scaled[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     # Fit the BaggingRegressor model
#     bagging_model.fit(X_train, y_train)

#     # Predictions on the test set
#     y_pred = bagging_model.predict(X_test)

#     # Compute metrics for this fold
#     fold_mae = mean_absolute_error(y_test, y_pred)
#     fold_mse = mean_squared_error(y_test, y_pred)

#     cv_mae.append(fold_mae)
#     cv_mse.append(fold_mse)

# # Calculate mean of metrics across folds
# mean_cv_mae = np.mean(cv_mae)
# mean_cv_mse = np.mean(cv_mse)

# print('Mean Cross-validated MAE:', mean_cv_mae)
# print('Mean Cross-validated RMSE:', np.sqrt(mean_cv_mse))

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_percentage_error
kf = KFold(n_splits=10, shuffle=True, random_state=69)
cv_mae = []
cv_mse = []
cv_mape = []
for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the BaggingRegressor model
    model.fit(X_train, y_train)

    # Predictions on the test set
    y_pred = model.predict(X_test)

    # Compute metrics for this fold
    fold_mae = mean_absolute_error(y_test, y_pred)
    fold_mse = mean_squared_error(y_test, y_pred)
    fold_mape = mean_absolute_percentage_error(y_test, y_pred)

    cv_mae.append(fold_mae)
    cv_mse.append(fold_mse)
    cv_mape.append(fold_mape)
# Calculate mean of metrics across folds
mean_cv_mae = np.mean(cv_mae)
mean_cv_mse = np.mean(cv_mse)
mean_cv_mape= np.mean(cv_mape)

print('Mean Cross-validated MAE:', mean_cv_mae)
print('Mean Cross-validated RMSE:', np.sqrt(mean_cv_mse))
print('Mean Cross-validated MAPE:', mean_cv_mape)